In [1]:
import instaloader
import pandas as pd
import datetime 

L = instaloader.Instaloader()
cutoffdate = datetime.datetime.today() - datetime.timedelta(days=2)

handles = ['uwengsoc','uwcsa','uw_ux','uwblueprint','uwaterlooeng','uwaterloottc','uwaterloodsc','uwaterloopm','uwmcc','gdscwaterloo','uwsmileclub','socratica.info','yourwusa','wataiteam','uwawscloud','techplusuw','itshera.co','uwstartups','electriummobility','uwhiphop','uwaterloo_ksa','uw_aviation','uwaterloopm','uwmcc','uwmsa','gdscwaterloo','waterloo_ultimate','uwcheeseclub','uwstreetdance','uwmidsun','watolink_uw','uwaterlooeng','uwpokerclub','uwaterloocycling','uwaterloobsa','uw_phys_club','uw.gsa','uwcsclub','uwfintech','uwaterloosc','uwactsciclub','uwstatsclub']
postscolumns = ['account','date','caption','accessibility_caption','hashtags']
postsDf = pd.DataFrame(columns = postscolumns)

cnt = 0

for handle in handles:
    profile = instaloader.Profile.from_username(L.context, handle)
    for post in profile.get_posts():
        if post.date > cutoffdate:
            if (post.accessibility_caption) == None:
                photo_caption = post.accessibility_caption
            else:
                photo_caption = post.accessibility_caption.replace('"','\"')
            new_row = pd.DataFrame({
                'id': cnt,
                'url': post.shortcode,
                'likes': post.likes,
                'display_photo': post.url,
                'account': [handle.replace('"','\"')],
                'date': [post.date],
                'caption': [post.caption.replace('"','\"')],
                'accessibility_caption': [photo_caption],
            })
            cnt += 1
            postsDf = pd.concat([postsDf, new_row], ignore_index=True)
        else:
            break  

print(postsDf)


C:\Users\david\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\win_unicode_console\__init__.py:31: RuntimeWarning: sys.stdin.encoding == 'utf-8', whereas sys.stdout.encoding == 'UTF-8', readline hook consumer may assume they are the same
  readline_hook.enable(use_pyreadline=use_pyreadline)
C:\Users\david\AppData\Local\Temp\ipykernel_8904\3213708331.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  postsDf = pd.concat([postsDf, new_row], ignore_index=True)


        account                date  \
0  uwaterloottc 2024-08-02 18:25:43   
1   uwsmileclub 2024-08-02 00:43:14   
2      yourwusa 2024-08-02 18:21:03   
3    uwawscloud 2024-08-01 03:36:06   
4    uwawscloud 2024-08-01 02:14:55   
5    techplusuw 2024-08-01 06:52:22   
6    itshera.co 2024-08-02 17:00:23   
7      uwcsclub 2024-08-02 05:19:44   

                                             caption  \
0  Hello everyone, we are happy to announce that ...   
1  Hey Smilees! 🌼 We’re hiring for the Fall 2024!...   
2  🚨 CONTEST ALERT 🚨 \n\nZeus is about to buckle ...   
3  🚀🌟 Experience a burst of creativity during our...   
4  🌟 Calling all student innovators! \n\n☁️ The S...   
5  Want to help create a more inclusive UW tech c...   
6  🌟✨ Fascinating Friday Feature! ✨🌟\n\nWhile Tik...   
7  ‼️Applications Extended ‼️\n\n❓Missed out on y...   

                               accessibility_caption hashtags   id  \
0  Photo by University of Waterloo Table Tennis C...      NaN  0.0   
1  

In [2]:
postsDf.reset_index(drop=True) 
postsDf.to_csv("instagram_raw.csv")

In [3]:
from together import Together
import os
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import json
import pandas as pd
from enum import Enum, auto
import re

load_dotenv()
togetherAPI = os.getenv('TOGETHER_API')
client = Together(api_key=togetherAPI)

basePrompt = open("C:/Users/david/Desktop/uw-upnext/WebScraper/basePrompt.in","r", encoding = "utf-8").read()

postsDf = pd.read_csv("C:/Users/david/Desktop/uw-upnext/WebScraper/instagram_raw.csv")

processedjson = []

for index, row in postsDf.iterrows():
    currentjson = f'"id": "{int(row["id"])}"|* "account": "{row["account"]}"|* "date": "{row["date"]}"|* "caption": "{row["caption"]}"|* "photo_caption": "{row["accessibility_caption"]}"'
    post = "{" + ',\n'.join(x for x in currentjson.replace('\n','\\n').split('|*')) + '}' 
    processedjson.append(post)

print("info loaded")


def remove_emojis(text):
    # Unicode ranges for emojis
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', text)

class Category(Enum):
    TECH = auto()
    DESIGN = auto()
    MUSIC = auto()
    ENTERTAINMENT = auto()
    CULTURE = auto()
    SPORTS = auto()
    WELLNESS = auto()
    GAMING = auto()

class Event(BaseModel):
    return_id: str = Field(description="return id of event")
    is_event: bool = Field(description="if the post contains an event")
    event_name: str = Field(description="name of event")
    event_description: str = Field(description='concise 20 word summary of what the event is, skip time or location')
    event_categories: str = Field(description='Categorize into one of the following: TECH, DESIGN, MUSIC, ENTERTAINMENT, CULTURE, SPORTS, WELLNESS, GAMING')
    start_time: str = Field(description="start time of event")
    end_time: str = Field(description="end time of event")
    location: str = Field(description= "location of event")



def return_event_details(inputJson : str):
    chat_completion = client.chat.completions.create(
    model = "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    response_format={"type": "json_object", "schema": Event.model_json_schema()},
        messages=[
            {
                "role": "system",
                "content": basePrompt,
            },
            {
                "role": "user",
                "content": "\n\n Input " + remove_emojis(inputJson) + "\n\n" + "Output",
            },
        ],
    )
    created_event = json.loads(chat_completion.choices[0].message.content)
    return created_event

def extract_details_with_error_handling(inputJson, index):
        try: 
            created_event = return_event_details(inputJson)
            return created_event
        except Exception as err:
            print(str(err))
            return {'return_id': index, 'is_event': False, 'event_name': None, 'start_time': None, 'end_time': None, 'location': None}
    
# print(extract_details_with_error_handling(processedjson[0], 0))

info loaded


In [4]:
import time
postsDf = pd.read_csv("instagram_raw.csv")

postsDf["is_event"] = pd.NA
postsDf["event_details"] = pd.NA

for index, row in postsDf.iterrows():
    time.sleep(5)
    event_details = extract_details_with_error_handling(processedjson[index],index)
    print(event_details)
    if event_details["event_name"] == None or event_details["start_time"] == None or event_details["end_time"] == None:
        postsDf.at[index, "is_event"] = False
        postsDf.at[index, "event_details"] = None
    else:
        postsDf.at[index, "is_event"] = True
        postsDf.at[index, "event_details"] = event_details

Error code: 520 - Error code: 520 -<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>


<title>api.together.xyz | 520: Web server is returning an unknown error</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/main.css" />


</head>
<body>
<div id="cf-wrapper">
    <div id="cf-error-details" class="p-0">
        <header class="mx-auto pt-10 lg:pt-6 lg:px-8 w-240 lg:w-full mb-8">
            <h1 class="inline-block sm:block sm:mb-2 

KeyboardInterrupt: 

In [5]:
postsDf.reset_index(drop=True) 
postsDf.to_csv("information_for_mongo.csv")

print(postsDf)

    Unnamed: 0.1  Unnamed: 0         account                 date  \
0              0           0           uw_ux  2024-07-21 14:24:54   
1              1           1           uw_ux  2024-07-20 03:02:24   
2              2           2     uwblueprint  2024-07-28 15:28:07   
3              3           3     uwblueprint  2024-07-26 23:21:02   
4              4           4     uwblueprint  2024-07-21 01:07:58   
5              5           5     uwblueprint  2024-07-18 14:44:10   
6              6           6     uwblueprint  2024-07-18 14:42:39   
7              7           7     uwblueprint  2024-07-18 14:38:51   
8              8           8     uwblueprint  2024-07-18 14:37:07   
9              9           9     uwblueprint  2024-07-18 14:29:24   
10            10          10    uwaterloottc  2024-07-26 14:53:39   
11            11          11    uwaterloottc  2024-07-25 02:44:06   
12            12          12    uwaterloodsc  2024-07-27 14:04:14   
13            13          13    uw